# Preprocessing

## Libraries

In [1]:
import pandas as pd
import re
from collections import Counter
from tqdm import tqdm

## Import data

In [3]:
bottins = pd.read_csv("data/strict_addressing.csv")
voies = pd.read_csv("data/opendata_voie_paris.csv", sep=";")
#keep only columns that might be useful further on
voies = voies[["N_SQ_VO", "L_VOIE", "L_COURTMIN", "L_LONGMIN", "Geometry", "LENGTH", "Geometry X Y"]]

## Preprocessing: replace special characters

In [6]:
# function to replace different characters at once in a string, based on a dictionary
def replace_chars_by_dict(string, map_dict):
    # DONT CHANGE NAME -> referenced in voies_merging.ipynb
    #https://www.geeksforgeeks.org/python-replace-different-characters-in-string-at-once/
    return re.compile("|".join(map_dict.keys())).sub(lambda ele: map_dict[re.escape(ele.group(0))], string)

def preprocess(df, column, map_dict):
    # DONT CHANGE NAME -> referenced in voies_merging.ipynb
    data = df.copy()
    new_colname = f"{column}_prep"
    data[new_colname] = data[column].str.lower()
    data[new_colname] = data[new_colname].apply(replace_chars_by_dict, args=(map_dict,))
    return data

In [19]:
# referenced in voies_merging.ipynb
map_dict = {"é": "e", "è": "e", "ê":"e", "à":"a", "â":"a", "ô":"o", "î":"i", "û":"u", "ç":"c", "\-":" ", "\_":" "}

In [22]:
preprocess(voies, "L_LONGMIN", map_dict).head(10)

,N_SQ_VO,L_VOIE,L_COURTMIN,L_LONGMIN,Geometry,LENGTH,Geometry X Y,L_LONGMIN_prep
0,750004016,BAUDRAN,Imp. Baudran,Impasse Baudran,"{""coordinates"": [[2.3554518641916804, 48.82415...",64.625200,"48.82415332715383,2.3558919735528483",impasse baudran
1,750003821,CHEVALIERS,Imp. des Chevaliers,Impasse des Chevaliers,"{""coordinates"": [[2.3960916096233094, 48.87258...",77.423197,"48.87229377669366,2.3963259349268453",impasse des chevaliers
2,750005946,BONS VIVANTS,R. des Bons Vivants,Rue des Bons Vivants,"{""coordinates"": [[2.346752283163513, 48.862212...",39.406151,"48.862044141078364,2.34682583693024",rue des bons vivants
3,750001312,JEAN HUGUES,R. Jean Hugues,Rue Jean Hugues,"{""coordinates"": [[2.274705254077503, 48.868429...",45.962263,"48.86861817275934,2.2748341164181163",rue jean hugues
4,750005774,JEAN-JACQUES ROUSSEAU,R. Jean-Jacques Rousseau,Rue Jean-Jacques Rousseau,"{""coordinates"": [[[2.3396869374353004, 48.8620...",397.842033,"48.86347293952501,2.342392728552715",rue jean jacques rousseau
5,750001077,MONTMORENCY,Bd. de Montmorency,Boulevard de Montmorency,"{""coordinates"": [[[2.260367904207046, 48.84827...",1000.487036,"48.851807735728976,2.262259841698861",boulevard de montmorency
6,750001758,COLONEL BONNET,Av. du Colonel Bonnet,Avenue du Colonel Bonnet,"{""coordinates"": [[2.2792145248001967, 48.85472...",183.462914,"48.855354120723064,2.2784143657170306",avenue du colonel bonnet
7,750002426,JEAN ARP,R. Jean Arp,Rue Jean Arp,"{""coordinates"": [[2.3738128128732803, 48.83645...",80.667420,"48.836737381917324,2.373471594916392",rue jean arp
8,750006314,VEGA,R. de la Véga,Rue de la Véga,"{""coordinates"": [[2.4040614336702038, 48.83647...",430.829662,"48.83816285893202,2.4048146289692838",rue de la vega
9,750001441,GEORGES LAFENESTRE,Av. Georges Lafenestre,Avenue Georges Lafenestre,"{""coordinates"": [[[2.3084150442511993, 48.8236...",445.971614,"48.82468410154668,2.310342687233298",avenue georges lafenestre
